In [31]:
import os
import random
import cv2
import numpy as np
from scipy import misc
from sklearn.decomposition import PCA

In [32]:
# This section of the program assigns a input directory and output directory  
# then the program selects a random leaf from the input directory and then randomly selects transformation to perform
# on the selected image, then saves the transformed image to a new directory. 

input_dir = 'AgandBio\leaves'
output_dir = 'Transformed_leaves'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i in range(100):
    # Chosing a random leaf image from the input directory 
    filename = random.choice(os.listdir(input_dir))
    filepath = os.path.join(input_dir, filename)
    img = cv2.imread(filepath)
    
    # Creating a varibale to choose a random transformation 
    transformation = np.random.choice(['rotate', 'shift', 'scale', 'warp'])
    
    # The below code applies the randomly selected transformation to the leaf.
    if transformation == 'rotate':
        angle = random.randint(-45, 45)

        rows, cols, _ = img.shape
        M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
        transformed_img = cv2.warpAffine(img, M, (cols, rows))
        
    elif transformation == 'scale':
        
        scale_factor = random.uniform(0.5, 2.0)
        
        rows, cols, _ = img.shape
        new_rows = int(rows * scale_factor)
        new_cols = int(cols * scale_factor)
        transformed_img = cv2.resize(img, (new_cols, new_rows))
        
    elif transformation == 'shift':
        dx = random.randint(-50, 50)
        dy = random.randint(-50, 50)

        
        rows, cols, _ = img.shape
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        transformed_img = cv2.warpAffine(img, M, (cols, rows))
        
    else:
       
        rows, cols, _ = img.shape
        src_pts = np.float32([[0, 0], [cols, 0], [cols, rows], [0, rows]])
        dst_pts = np.float32([[random.randint(0, cols//2), random.randint(0, rows//2)],
                              [random.randint(cols//2, cols), random.randint(0, rows//2)],
                              [random.randint(cols//2, cols), random.randint(rows//2, rows)],
                              [random.randint(0, cols//2), random.randint(rows//2, rows)]])
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)

    
        transformed_img = cv2.warpPerspective(img, M, (cols, rows))
        

    # Saving the transformed imaged to the transformed_leaves directory 
    output_path = os.path.join(output_dir, f'transformed_{i}.jpg')
    cv2.imwrite(output_path, transformed_img)

In [33]:
# This section of the program assigns a input directory and output directory  
# then the program reads the tranformed images of the leaves and creates patches and saves them in a new directory

input_dir2 = 'Transformed_leaves'
output_dir2 = 'Patches_of_Transformed_leaves'

if not os.path.exists(output_dir2):
    os.makedirs(output_dir2)
# This section of the program reads the tranformed images of the leaves and creates patches and saves them in a new directory

files = os.listdir(input_dir2)
for file in files:
    # Reading the Transformed leaf images from the Transformed_leaves directory
    img = cv2.imread(os.path.join(input_dir2, file))

    # Getting the shape of the images of the transformed leaves
    height, width, channels = img.shape

    # defining a patch size and stride.  
    patch_size = 128
    stride = 64

    for y in range(0, height-patch_size+1, stride):
        for x in range(0, width-patch_size+1, stride):
            
            patch = img[y:y+patch_size, x:x+patch_size]

            # Saving patches images to the Patches_of_transformed_leaves directory
            patch_filename = os.path.splitext(file)[0] + f"_patch_{y}_{x}.jpg"
            cv2.imwrite(os.path.join(output_dir2, patch_filename), patch)

In [34]:
# This section of the program assigns a input directory and output directory  
# then the program reads the patches of the transformed images and performs ZCA prewhittening 

input_dir3 = 'Patches_of_Transformed_leaves'
output_dir3 = 'Prewhitened_Patches'

if not os.path.exists(output_dir3):
    os.makedirs(output_dir3)

files = os.listdir(input_dir3)
for file in files:

    patch = cv2.imread(os.path.join(input_dir3, file))

    # converting patches to grayscale 
    patch = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    # Convert the patch to a float32 array
    patch = patch.astype(np.float32)

    # Centering the input datat by substracting the mean of the data from each data point
    mean = np.mean(patch)
    patch -= mean

    # Calculating covariance matrix of the centered data
    cov = np.dot(patch.T, patch) / patch.shape[0]

    # calculating eigenvalues and eigenvectors of the covariance matrix
    u, s, v = np.linalg.svd(cov)

    # ZCA matrix calculation. Chosing epsilon as 0.1. Stanford link says to choose o.01, 0.1 or 1.
    epsilon = 0.01
    zca_matrix = np.dot(u, np.dot(np.diag(1.0 / np.sqrt(s + epsilon)), u.T))

    # applying ZCA whitening transform to patches
    patch_zca = np.dot(patch, zca_matrix)

    # Rescaling the patch to 0-255 and converting back to uint8 to ensure pixel values are integer
    patch_zca_rescaled = (patch_zca - np.min(patch_zca)) / (np.max(patch_zca) - np.min(patch_zca)) * 255
    patch_zca_uint8 = patch_zca_rescaled.astype(np.uint8)

    # Saving to output directory
    patch_filename = os.path.splitext(file)[0] + '_prewhitened.jpg'
    cv2.imwrite(os.path.join(output_dir3, patch_filename), patch_zca_uint8)


C:\Users\mbshoa\AppData\Local\Temp\ipykernel_23848\282342709.py:39: RuntimeWarning: invalid value encountered in true_divide
  patch_zca_rescaled = (patch_zca - np.min(patch_zca)) / (np.max(patch_zca) - np.min(patch_zca)) * 255
